##### Installing libraries

In [15]:
!pip install gspread
!pip install oauth2client
!pip install google-api-python-client
!pip install pandas

You should consider upgrading via the '/home/iazzari/miniconda3/envs/investment_scripts/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/iazzari/miniconda3/envs/investment_scripts/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/iazzari/miniconda3/envs/investment_scripts/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/home/iazzari/miniconda3/envs/investment_scripts/bin/python -m pip install --upgrade pip' command.


##### Calling libraries

In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pandas as pd

##### Functions

In [17]:
SCOPE = ['https://spreadsheets.google.com/feeds']

def getSheetCredentials(pathAPIJson):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

    creds = ServiceAccountCredentials.from_json_keyfile_name(pathAPIJson, scope)

    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    
    return sheet

def numberToLetters(q):
    q = q - 1
    result = ''
    while q >= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

def getSpreadsheet(sheet, spreadsheetId, sheetName):
    result = sheet.values().get(spreadsheetId=spreadsheetId,
                                    range=sheetName+"!A1:Z1000").execute()
    values = result.get('values', [])

    dataframe = pd.DataFrame(values[1:], columns=values[0])

    return dataframe

def insertSpreadsheet(sheet, df):

    columns = df.columns.values.tolist()
    # selection of the range that will be updated
    cellList = sheet.range('A1:'+numberToLetters(len(columns))+'1')
    # modifying the values in the range
    for cell in cellList:
        val = columns[cell.col-1]
        if type(val) is str:
            val = val
        cell.value = val
    # update in batch
    sheet.update_cells(cellList)

    ############## VALUES ##############
    # number of lines and columns
    numLines, numColumns = df.shape
    # selection of the range that will be updated
    cell_list = sheet.range('A2:'+ numberToLetters(numColumns)+str(numLines+201))
    # modifying the values in the range
    for cell in cellList:
        try:
            val = df.iloc[cell.row-2,cell.col-1]
            if type(val) is str:
                val = val

        except:
            val = ""
        cell.value = val
    # update in batch
    sheet.update_cells(cellList)
    
def clearSpreadsheet(sheet, nLin, nCol):
    # selection of the range that will be updated
    cellList = sheet.range('A1:'+numberToLetters(nCol)+str(nLin))
    # modifying the values in the range
    for cell in cellList:
        val=""
        cell.value = val
    # update in batch
    sheet.update_cells(cellList)

##### Examples

In [18]:
#get sheet credentials
sheet = getSheetCredentials('<path_to_sheet_credential>/sheet_credential.json')

#using my sheet: https://docs.google.com/spreadsheets/d/1MdwfFFObicVn9KXEJHgtFL40xgMoWxs-LNQAxxzY4bw/edit#gid=0

#get df of sheet values 
df = getSpreadsheet(sheet, '1MdwfFFObicVn9KXEJHgtFL40xgMoWxs-LNQAxxzY4bw', 'new_API')

df

,TICKER,TIPO DE FUNDO,INADIMPLÊNCIA,ABL,PATRIMÔNIO LÍQUIDO,DY ACUMULADO (12 MESES),VALOR PATRIMONIAL,P/VPA,VACÂNCIA FÍSICA,VALOR DE MERCADO,VOLUME,VACÂNCIA FINANCEIRA,SEGMENTO,ADMINISTRADOR,QUANTIDADE DE IMÓVEIS,NOME,PÚBLICO ALVO,DY MÉDIO (12 MESES),data_ultima_modificacao
0,FPNG11,,,,65301479.13,,102.68,0.97,,,10.00,,Lajes Corporativas,Unitas,0.00,FII P NEGRA,Investidores em Geral,,01-09-2020
1,RCFA11,,,,18757943.54,,1.61,0.56,,10510679.70,20302.00,,Híbrido,FRAM Capital,0.00,FII GP RCFA,Investidor Profissional,,01-09-2020
2,FINF11,,,,105052992.37,,146.41,,,,,,Logística,Planner,0.00,FII INFRA RE,Investidor Qualificado,,01-09-2020
3,NCHB11B,,,,17081590.66,,416.62,,,,,,Títulos e Val Mob,BTG Pactual,0.00,FII NCH BRA,Investidor Qualificado,,01-09-2020
4,BZLI11,,,,428081706.75,0.00,11.86,1.37,,588105450.70,20065.00,,Títulos e Val Mob,Foco Investimentos,0.00,FII BRREALTY,Investidor Qualificado e Profissional,0.00,01-09-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,FIIP11B,Fundo de Tijolo,,69032.09,183549267.97,11.91,197.97,0.97,0.00,177996560.76,1140.00,,Logística,Oliveira Trust,7.00,FII RB CAP I,Investidores em Geral,0.99,01-09-2020
248,RBGS11,Fundo de Tijolo,,12769.92,93359985.81,4.60,86.60,0.46,13.50,42796600.00,985.00,,Shoppings,Oliveira Trust,1.00,FII RB GSB I,Investidores em Geral,0.38,01-09-2020
249,CEOC11,Fundo de Tijolo,,11942.00,178069316.36,7.61,98.07,0.83,0.00,148560246.72,1877.00,,Lajes Corporativas,BTG Pactual,1.00,FII CEO CCP,Investidores em Geral,0.63,01-09-2020
250,SAAG11,Fundo de Tijolo,,,627916463.29,8.58,111.50,0.96,,600375494.32,12868.00,,Outros,Rio Bravo,0.00,FII SANT AGE,Investidores em Geral,0.72,01-09-2020
